In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TFAutoModelForSequenceClassification

In [3]:
dataset_train = load_dataset('csv', data_files='feedback-prize-effectiveness/train.csv')
def map_label(example):
    mapping = {'Ineffective': 1, 'Adequate': 2, 'Effective': 3}
    return {"labels":mapping[example['discourse_effectiveness']]}

Using custom data configuration default-6be47e64410e8e15
Reusing dataset csv (C:\Users\joshz\.cache\huggingface\datasets\csv\default-6be47e64410e8e15\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset_train = dataset_train.map(map_label)
# dataset_train['labels'] = pd.Series(dataset_train['train']['discourse_effectiveness']).map(
#     {'Ineffective': 1, 'Adequate': 2, 'Effective': 3}).tolist()


Parameter 'function'=<function map_label at 0x0000023A993E9EE8> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/36765 [00:00<?, ?ex/s]

In [5]:
dataset_train['train'][0]

{'discourse_id': '0013cc385424',
 'essay_id': '007ACE74B050',
 'discourse_text': "Hi, i'm Isaac, i'm going to be writing about how this face on Mars is a natural landform or if there is life on Mars that made it. The story is about how NASA took a picture of Mars and a face was seen on the planet. NASA doesn't know if the landform was created by life on Mars, or if it is just a natural landform. ",
 'discourse_type': 'Lead',
 'discourse_effectiveness': 'Adequate',
 'labels': 2}

In [6]:

dataset = dataset_train['train'].train_test_split(train_size=0.8)

In [7]:
dataset['train'][1233]

{'discourse_id': 'dde0981ed2dd',
 'essay_id': '9329A9DD13F2',
 'discourse_text': 'Online classes will allow students who have frequent absences to attend class and make up work with more ease ',
 'discourse_type': 'Claim',
 'discourse_effectiveness': 'Effective',
 'labels': 3}

In [8]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_projector', 'activation_13', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [9]:
model

In [10]:
def tokenize_function(example):
    return tokenizer(example["discourse_text"], truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['discourse_id', 'essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 29412
    })
    test: Dataset({
        features: ['discourse_id', 'essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 7353
    })
})

In [11]:

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [12]:
tokenized_datasets['train'].column_names

['discourse_id',
 'essay_id',
 'discourse_text',
 'discourse_type',
 'discourse_effectiveness',
 'labels',
 'input_ids',
 'attention_mask']

In [13]:

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=['labels'],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [14]:
type(tf_train_dataset)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [15]:

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [16]:
tf_train_dataset

<PrefetchDataset shapes: ({input_ids: (None, None), attention_mask: (None, None)}, (None,)), types: ({input_ids: tf.int64, attention_mask: tf.int64}, tf.int64)>

In [18]:

model.fit(tf_train_dataset,validation_data=tf_test_dataset,epochs=5,verbose=1,steps_per_epoch=200)

Epoch 1/5
 15/200 [=>............................] - ETA: 18s - loss: nan - accuracy: 0.0000e+00

ResourceExhaustedError:  OOM when allocating tensor with shape[8,423,12,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node tf_distil_bert_for_sequence_classification/distilbert/transformer/layer_._5/attention/transpose_3 (defined at \miniconda3\envs\kaggle-feedback\lib\site-packages\transformers\models\distilbert\modeling_tf_distilbert.py:183) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_13048]

Errors may have originated from an input operation.
Input Source operations connected to node tf_distil_bert_for_sequence_classification/distilbert/transformer/layer_._5/attention/transpose_3:
 tf_distil_bert_for_sequence_classification/distilbert/transformer/layer_._5/attention/MatMul_1 (defined at \miniconda3\envs\kaggle-feedback\lib\site-packages\transform

In [ ]:
tf.test.is_gpu_available()

In [89]:
dataset_train = load_dataset('csv', data_files='feedback-prize-effectiveness/train.csv')

dataset_train['labels'] = pd.Series(dataset_train['train']['discourse_effectiveness']).map({'Ineffective': 1, 'Adequate': 2, 'Effective':3}).tolist()


Using custom data configuration default-541aa7b1f33b3777
Reusing dataset csv (C:\Users\joshz\.cache\huggingface\datasets\csv\default-541aa7b1f33b3777\0.0.0\51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58)


  0%|          | 0/1 [00:00<?, ?it/s]

In [90]:

dataset = dataset_train['train'].train_test_split(train_size=0.8)

In [91]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_159']
You should probably TRAIN this model on a down-stream task to be able to use 

In [92]:
def tokenize_function(example):
    return tokenizer(example["discourse_text"],  truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/30 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['discourse_id', 'essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'input_ids', 'attention_mask'],
        num_rows: 29412
    })
    test: Dataset({
        features: ['discourse_id', 'essay_id', 'discourse_text', 'discourse_type', 'discourse_effectiveness', 'input_ids', 'attention_mask'],
        num_rows: 7353
    })
})

In [99]:

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [100]:

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=['labels'],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [101]:

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [102]:
tf_train_dataset

<PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, {})>

In [103]:

model.fit(
    tf_train_dataset,
)

TypeError: in user code:

    File "C:\Users\joshz\miniconda3\envs\Feedback-Prize\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\joshz\miniconda3\envs\Feedback-Prize\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\joshz\miniconda3\envs\Feedback-Prize\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\joshz\miniconda3\envs\Feedback-Prize\lib\site-packages\transformers\modeling_tf_utils.py", line 1154, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "C:\Users\joshz\miniconda3\envs\Feedback-Prize\lib\site-packages\keras\optimizers\optimizer_v2\optimizer_v2.py", line 537, in minimize
        grads_and_vars = self._compute_gradients(
    File "C:\Users